# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [ ]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# UnderSampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
print(y_data)
y = y_data.values.reshape(-1, 1)
print(y)

X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

# Type value 

In [ ]:
b_Types = {'I':0, 'E':1, 'N':0, "S":1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Types_list =[{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    return[b_Types[i] for i in personality]

def translate_back(personality):
    s = ""
    for i, j in enumerate(personality):
        s += b_Types_list[i][j]
    return s

d = y[0]
print(d)
list_personality_b = np.array([translate_personality(p) for p in d])
print(list_personality_b)

# tokenize and stemming

In [ ]:
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

In [ ]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])
stemData = np.array(stemDaa)
#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

# Vectorization

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
count_vectorizer = CountVectorizer(
    analyzer="word",
    max_features=1500,
    tokenizer=None,
    stop_words=None,
    max_df=0.7,
    min_df=0.1
)

#X_ct = count_vectorizer.fit_transform(stemData)

def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
print(X)
y = np.array(y_data)

# XGBoost_Undersampling

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)


# Model Building & Evaluation

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [ ]:
print("Model building Start")
print(max(len(l) for l in X_train)
model = XGBClassifier()
model.fit(X_train, y_train, verbose=1)
#model.get_booster().get_score(importance_type="gain") #변수의 중요도

In [ ]:
import xgboost as xgb
#test_hp = xgb.DMatrix(data=VHX_data, label=hy)
#preds = model.predict(X_test)
print("Model building End")

In [ ]:
import joblib

joblib.dump(model, 'XGBoost_undersample.pkl')

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

# Harry Potter

In [2]:
import nltk
import numpy as np
import pandas as pd

from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
import glob   

'''HP data Normalization and Stemming'''
# bag_of_words = {} #{등장인물 : 해당인물의 stemData, ~}
h_normalized_text = []
h_stemData = []

ps=PorterStemmer()
stop_words = set(stopwords.words('english'))
input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

harryPotter={}
for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    harryPotter[df['Character'][0]] = df['Sentence'].ravel()

print("stemming 시작")
for element in harryPotter.keys():
    for sentence in harryPotter[element]:
        h_clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
        h_normalized_text.append(h_clean_sentence)
        for sentence in h_normalized_text:
            tokenData = nltk.word_tokenize(sentence)
            for word in tokenData:
                if word not in stop_words:
                    h_stemData.append(ps.stem(word))
    harryPotter[element] = h_stemData

print("stemming 완료")

['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']
stemming 시작
stemming 완료


In [3]:
'''HP data Vectorization'''
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer

stemData = []
hy=[]
for element in harryPotter.keys():
    stemData.append(harryPotter[element])
    hy.append(element)
HX_data = tfidf.transform(stemData)
print(HX_data)
HY_data = np.array(hy)

'''for element in harryPotter.keys():
    tfidf.fit(harryPotter[element])
    script = tfidf.transform(harryPotter[element])
    X.append(script)
    y.append(element)

print(X.shape)'''

  (0, 1499)	7.338210456553937e-06
  (0, 89)	7.338210456553937e-06
  (0, 1154)	1.7122491065292518e-05
  (0, 169)	4.158319258713898e-05
  (0, 346)	0.00010273494639175512
  (0, 733)	7.093603441335472e-05
  (0, 1365)	7.093603441335472e-05
  (0, 36)	7.827424486990865e-05
  (0, 1372)	9.784280608738582e-05
  (0, 596)	0.00010028887623957047
  (0, 1495)	0.0001614406300441866
  (0, 1310)	0.00029597448841434213
  (0, 783)	0.00016388670019637126
  (0, 133)	0.00016388670019637126
  (0, 554)	0.00017367098080510983
  (0, 228)	0.00017611705095729448
  (0, 825)	0.00017611705095729448
  (0, 1441)	0.00018590133156603307
  (0, 1195)	0.000193239542022587
  (0, 857)	0.00019568561217477164
  (0, 1393)	0.00020057775247914094
  (0, 43)	0.0002030238226313256
  (0, 1467)	0.0002030238226313256
  (0, 1463)	0.0002030238226313256
  (0, 128)	0.0002030238226313256
  :	:
  (4, 443)	0.007325980105793013
  (4, 603)	0.06672390161129277
  (4, 416)	0.04377487144349642
  (4, 437)	0.02377824794938694
  (4, 962)	0.032383522744

'for element in harryPotter.keys():\n    tfidf.fit(harryPotter[element])\n    script = tfidf.transform(harryPotter[element])\n    X.append(script)\n    y.append(element)\n\nprint(X.shape)'

# Predict & Get Results

In [12]:
import joblib  
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

model = joblib.load('XGBoost_undersample.pkl')
# xgb_model_loaded.get_xgb_params()    # Get xgboost specific parameters.
harrypotter_predict = model.predict(HX_data)
print(harrypotter_predict)

['ESFP' 'ESFP' 'ESFP' 'ESFP' 'ESFP']


In [13]:
cols_when_model_builds = model.get_booster().feature_names
len(cols_when_model_builds)

38863